#Assessment 2: Building a Knowledge Graph with Semantic Search and Query Answering
Instructions:
- Build a basic RDF knowledge graph using the rdflib library.
- Example: "Alice is a person," "Alice knows Bob," "Bob works at Acme Corp."
- Convert the RDF triples into simple text sentences (e.g., "Alice knows Bob").
- Embed these text sentences using any method you prefer (e.g., sentence-transformers, HuggingFace, or other embedding techniques).
- Store the embeddings in FAISS or ChromaDB.
- Accept a simple user query like: "Who does Alice know?" and return the relevant text from the stored embeddings.

In [1]:
!pip install rdflib sentence-transformers chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.2/587.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 17.8 MB/s e

In [2]:
# Build basic RDF
from rdflib import Graph, Namespace, RDF

g = Graph()
ex = Namespace("http://example.org/")

g.add((ex.Alice, RDF.type, ex.Person))
g.add((ex.Alice, ex.knows, ex.Bob))
g.add((ex.Bob, ex.worksAt, ex.AcmeCorp))


<Graph identifier=Ncf380227d46f4fec8bdaf4ba29585834 (<class 'rdflib.graph.Graph'>)>

In [3]:
# Convert RDF triples into simple text sentences
sentences = [
    "Alice is a person",
    "Alice knows Bob",
    "Bob works at Acme Corp"
]


In [5]:
# create Embeddings
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
sentence_embeddings = model.encode(sentences)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# Store embeddings in ChromaDB
import chromadb

client = chromadb.Client()
collection = client.create_collection(name="knowledge_graph1")

for i in range(len(sentence_embeddings)):
    collection.add(
        ids=[str(i)],
        embeddings=[sentence_embeddings[i]],
        documents=[sentences[i]]
    )


In [9]:
# Query
query = "Who does Alice know"
query = query.lower()

query_embedding = model.encode([query])


In [10]:
# Retrive
result = collection.query(
    query_embeddings=query_embedding,
    n_results=1
)

print("Answer:", result["documents"][0][0])


Answer: Alice knows Bob
